# EWM(지수이동평균) 직접 만들기


In [3]:
import FinanceDataReader as fdr

df = fdr.DataReader("005930", start="2021")

df["Close"]

Date
2021-01-04     83000
2021-01-05     83900
2021-01-06     82200
2021-01-07     82900
2021-01-08     88800
               ...  
2026-01-26    152100
2026-01-27    159500
2026-01-28    162400
2026-01-29    160700
2026-01-30    160500
Name: Close, Length: 1246, dtype: int64

In [8]:
df = df[["Close"]]
df

,Close
Date,
2021-01-04,83000
2021-01-05,83900
2021-01-06,82200
2021-01-07,82900
2021-01-08,88800
...,...
2026-01-26,152100
2026-01-27,159500
2026-01-28,162400


In [16]:
N = 9
weight = 2 / (1 + N)
print(weight)

ewm_values = [float(df["Close"].iloc[0])]
for index in range(len(df["EWM"]) - 1):
    val1 = df["Close"].iloc[index + 1] * weight
    val2 = ewm_values[index] * (1 - weight)
    ewm_values.append(val1 + val2)

df["EWM"] = ewm_values
df["EWM"]

0.2


Date
2021-01-04     83000.000000
2021-01-05     83180.000000
2021-01-06     82984.000000
2021-01-07     82967.200000
2021-01-08     84133.760000
                  ...      
2026-01-26    147786.451998
2026-01-27    150129.161598
2026-01-28    152583.329278
2026-01-29    154206.663423
2026-01-30    155465.330738
Name: EWM, Length: 1246, dtype: float64

In [17]:
def ewm_fn(df, window):
    df = df.copy()

    weight = 2 / (1 + window)

    ewm_values = [float(df["Close"].iloc[0])]
    for index in range(len(df["EWM"]) - 1):
        val1 = df["Close"].iloc[index + 1] * weight
        val2 = ewm_values[index] * (1 - weight)
        ewm_values.append(val1 + val2)

    df["EWM"] = ewm_values
    return df[["EWM"]]

In [18]:
df_ewm = ewm_fn(df, window=9)
df_ewm

,EWM
Date,
2021-01-04,83000.000000
2021-01-05,83180.000000
2021-01-06,82984.000000
2021-01-07,82967.200000
2021-01-08,84133.760000
...,...
2026-01-26,147786.451998
2026-01-27,150129.161598
2026-01-28,152583.329278


# EWM(지수이동평균)\_pandas.DataFrame.ewm


In [ ]:
import FinanceDataReader as fdr

df = fdr.DataReader("005930", start="2021")
df = df[["Close"]]  # 데이터 프레임으로 만들어주기 위해
df

,Close
Date,
2021-01-04,83000
2021-01-05,83900
2021-01-06,82200
2021-01-07,82900
2021-01-08,88800
...,...
2026-01-28,162400
2026-01-29,160700
2026-01-30,160500


In [5]:
short_N = 9
long_N = 26
signal_N = 13

df["Short"] = df["Close"].ewm(span=short_N, adjust=False).mean()  # 지수이동평균
df["Long"] = df["Close"].ewm(span=long_N, adjust=False).mean()
df["MACD"] = df["Short"] - df["Long"]
df["Signal"] = df["MACD"].ewm(span=signal_N, adjust=False).mean()
df["MACD Oscillator"] = df["MACD"] - df["Signal"]
df

,Close,Short,Long,MACD,Signal,MACD Oscillator
Date,,,,,,
2021-01-04,83000,83000.000000,83000.000000,0.000000,0.000000,0.000000
2021-01-05,83900,83180.000000,83066.666667,113.333333,16.190476,97.142857
2021-01-06,82200,82984.000000,83002.469136,-18.469136,11.239103,-29.708239
2021-01-07,82900,82967.200000,82994.878829,-27.678829,5.679398,-33.358228
2021-01-08,88800,84133.760000,83424.887805,708.872195,106.135512,602.736683
...,...,...,...,...,...,...
2026-01-28,162400,152583.329278,139006.214867,13577.114411,12200.497907,1376.616505
2026-01-29,160700,154206.663423,140613.161914,13593.501509,12399.498421,1194.003087
2026-01-30,160500,155465.330738,142086.261032,13379.069707,12539.437176,839.632530


# macdOscillator_fn 함수만들기


In [1]:
import FinanceDataReader as fdr

df = fdr.DataReader("005930", start="2021")
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691
2021-01-05,81600,83900,81600,83900,35335669,0.010843
2021-01-06,83300,84500,82100,82200,42089013,-0.020262
2021-01-07,82800,84200,82700,82900,32644642,0.008516
2021-01-08,83300,90000,83000,88800,59013307,0.071170
...,...,...,...,...,...,...
2026-01-29,166200,166600,157100,160700,36087223,-0.010468
2026-01-30,160100,166500,160100,160500,40557267,-0.001245
2026-02-02,155700,159600,150400,150400,39748369,-0.062928


In [2]:
def macdOscillator_fn(df, short_N, long_N, signal_N):
    df = df[["Close"]].copy()

    df["Short"] = df["Close"].ewm(span=short_N, adjust=False).mean()
    df["Long"] = df["Close"].ewm(span=long_N, adjust=False).mean()
    df["MACD"] = df["Short"] - df["Long"]
    df["Signal"] = df["MACD"].ewm(span=signal_N, adjust=False).mean()
    df["MACD Oscillator"] = df["MACD"] - df["Signal"]

    return df[["MACD", "Signal", "MACD Oscillator"]]

In [3]:
df_macd_oscillator = macdOscillator_fn(df, short_N=9, long_N=26, signal_N=13)
df_macd_oscillator

,MACD,Signal,MACD Oscillator
Date,,,
2021-01-04,0.000000,0.000000,0.000000
2021-01-05,113.333333,16.190476,97.142857
2021-01-06,-18.469136,11.239103,-29.708239
2021-01-07,-27.678829,5.679398,-33.358228
2021-01-08,708.872195,106.135512,602.736683
...,...,...,...
2026-01-29,13593.501509,12399.498421,1194.003087
2026-01-30,13379.069707,12539.437176,839.632530
2026-02-02,11750.171043,12426.684871,-676.513829
